# Combining and analysing the low density anglescan simulations for all 5 probe tips

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu

In [2]:
importlib.reload(spl)
importlib.reload(td)
importlib.reload(fd)
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [5]:
# lowdens_dir = pth.Path('/home/jleland/Spice/spice2/bin/data_local_m2/lowdens_anglescan')
lowdens_dir = pth.Path('/home/jleland/data/external/spice/')
os.chdir(lowdens_dir)

In [4]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [8]:
skippable_scans = {
    'marconi/lowdens_anglescan/inpartest_as',
    'marconi/lowdens_anglescan/angled_tiltscan_ext',
    'marconi/lowdens_anglescan/sprobe_tiltscan',
    'marconi/lowdens_anglescan/recessed_tiltscan',
    'marconi/lowdens_anglescan/rearwall_tiltscan',
    'marconi/lowdens_anglescan/rearwall_tiltscan_redo_1',
    'marconi/lowdens_anglescan/rearwall_tiltscan_redo.1',
}
single_sims = {
    'marconi/lowdens_anglescan/angled_1',
}

In [9]:
scans_searchstr = '*/lowdens_anglescan/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}

desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        print(f'Single sim {scan} found')
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
    print(f'{scan}: {all_run_dirs[scan]}\n')


marconi/lowdens_anglescan/rearwall_tiltscan_redo: ['marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-1.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-10.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-15.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-2.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-20.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-3.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-30.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-4.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-45.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-5.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-6.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-60.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-7.0', 'marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-75.0', 'marconi/lowdens_a

## Plotting all IV characterisitics on the same axes

In [10]:
skippable = {
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-3.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-1.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-1.0_restart',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-90.0',
    'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-90.0',
    'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-1.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-4.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-5.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-6.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-7.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-8.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-9.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-4.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-5.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-4.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-5.0',
    'marconi/lowdens_anglescan/sprobe_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/sprobe_tiltscan/alpha_yz_-10.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan_redo.1/alpha_yz_-1.0',
#     'marconi/lowdens_anglescan/sprobe_tiltscan_redo/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0'
    
#     'cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-1.0',
#     'cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0'
}

In [21]:
# fig, axes = plt.subplots(2)
splopters = {}
scans = list(scans)

errors = {
    'assertion': [],
    'type': [],
    'failed_sim': [],
    'value': [],
}

for i in range(len(scans)):
#     ax = axes[i]
    scan = scans[i]

    for angle_dir in all_run_dirs[scan]:
        print(f'\nCreating splopter for {scan}, {angle_dir}')
        if angle_dir in skippable:
            print('Skipping...')
            continue
        try:
            splopter = spl.Splopter(lowdens_dir / angle_dir, reduce=desired_variables)
            splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
    #         splopter.denormalise()
            splopters[angle_dir] = splopter
        except AssertionError as e:
            print(f'AssertionError encountered for {angle_dir}. Skipping...')
            errors['assertion'].append(angle_dir)
            continue
        except TypeError as e:
            print(f'TypeError encountered for {angle_dir}. Skipping...')
            errors['type'].append(angle_dir)
            continue
        except ValueError as e:
            print(f'valueException encountered for {angle_dir}. Skipping...')
            errors['value'].append(angle_dir)
            continue
        except spl.FailedSimulationException as e:
            print(f'FailedSimulationException encountered for {angle_dir}. Skipping...')
            errors['failed_sim'].append(angle_dir)
            continue
            
        
print('\n...Done')


Creating splopter for marconi/lowdens_anglescan/rearwall_tiltscan_redo, marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-1.0
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-1.0/backup_20200107-2128/t-alpha_yz_-1.0.mat
FailedSimulationException encountered for marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-1.0. Skipping...

Creating splopter for marconi/lowdens_anglescan/rearwall_tiltscan_redo, marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-10.0
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-10.0/backup_20200107-1018/t-alpha_yz_-10.0.mat
Loading backup backup_20200107-1018 (1 of 5) for current and bias concatenation
Detected finish, stripping tail for backup 1 of 5
Found an odd number of spike transitions

Creating splopter f

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-30.0/backup_20200107-0407/t-alpha_yz_-30.0.mat
Loading backup backup_20200107-0407 (1 of 5) for current and bias concatenation
Detected finish, stripping tail for backup 1 of 5
Found an odd number of spike transitions

Creating splopter for marconi/lowdens_anglescan/rearwall_tiltscan_redo, marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-4.0
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-4.0/backup_20200107-2128/t-alpha_yz_-4.0.mat
Loading backup backup_20200107-2128 (1 of 5) for current and bias concatenation
Loading backup backup_20200108-0936 (2 of 5) for current and bias concatenation
Detected finish, stripping tail for backup 2 of 5
Stripping head for backup 2 of 5
Found an odd number of spike tra

In [22]:
for i, (filename, splopter) in enumerate(splopters.items()):
    print(f'{i}) {filename}: {splopter}')

0) marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-10.0: <flopter.spice.splopter.Splopter object at 0x7f81506ca128>
1) marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-15.0: <flopter.spice.splopter.Splopter object at 0x7f81506c7358>
2) marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-2.0: <flopter.spice.splopter.Splopter object at 0x7f81516e7c50>
3) marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-20.0: <flopter.spice.splopter.Splopter object at 0x7f81541c50b8>
4) marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-3.0: <flopter.spice.splopter.Splopter object at 0x7f815218d9b0>
5) marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-30.0: <flopter.spice.splopter.Splopter object at 0x7f81505855c0>
6) marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-4.0: <flopter.spice.splopter.Splopter object at 0x7f81506d8b70>
7) marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-45.0: <flopter.spice.splopter.Splopter object at 0x7

In [23]:
# For adding a new run to the list of splopters
angle_dir = all_run_dirs[scans[6]][0]
print(angle_dir)

if angle_dir in splopters:
    print(f'Removing {angle_dir} from existing list of splopters')
    splopters.pop(angle_dir)
splopter = spl.Splopter(lowdens_dir / angle_dir, reduce=desired_variables)
splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
splopter.denormalise()
splopters[angle_dir] = splopter

marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-1.0
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-1.0/backup_20191126-1416/t-alpha_yz_-1.0.mat
Loading backup backup_20191126-1416 (1 of 5) for current and bias concatenation
Loading backup backup_20191127-1127 (2 of 5) for current and bias concatenation
Stripping head for backup 2 of 5
Loading backup backup_20191128-1528 (3 of 5) for current and bias concatenation
Stripping head for backup 3 of 5
Loading backup backup_20191128-1944 (4 of 5) for current and bias concatenation
Detected finish, stripping tail for backup 4 of 5
Stripping head for backup 4 of 5
Found an odd number of spike transitions


In [24]:
for error, simulations in errors.items():
    for simulation in simulations:
        print(f'{error}: \t {simulation}')

type: 	 marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-89.0
type: 	 marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-10.0
type: 	 marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-6.0
type: 	 marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-7.0
type: 	 marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-8.0
type: 	 marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-89.0
failed_sim: 	 marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-1.0
failed_sim: 	 cumulus/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-1.0
failed_sim: 	 cumulus/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-2.0
failed_sim: 	 cumulus/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-3.0
failed_sim: 	 cumulus/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-4.0
failed_sim: 	 cumulus/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-5.0


## Fitting and Adding Info to a Dataframe
Might be good to make it into a dataset instead so individual IV chracterisitcs can be stored as well.

In [15]:
axes = []
for i in range(5):
    fig, ax = plt.subplots(1, 3, sharex='col', sharey='col')
    axes.append(ax)

fit_df = pd.DataFrame(columns=['probe', 
                               'angle', 
                               'temp',
                               'd_temp',
                               'isat', 
                               'd_isat',
                               'a',
                               'd_a',
                               'v_f',
                               'd_v_f'
                              ])

for label, s in splopters.items():
    print('=== ' + label + ' ===')
#     if angle == 2.0:
#         continue
    
    if label in skippable:
        print(f'Skipping {label}')
        continue
    elif 'inpartest' in label or 'flush' in label:
        ax = axes[0][0]
        axt = axes[0][1]
        axd = axes[0][2]
        probe = 'flush'
        ax.set_title('Flush')
    elif 'angled' in label:
        ax = axes[1][0]
        axt = axes[1][1]
        axd = axes[1][2]
        probe = 'angled'
        ax.set_title('Angled Tip')
    elif 'rearwall' in label:
        ax = axes[2][0]
        axt = axes[2][1]
        axd = axes[2][2]
        probe = 'rearwall'
        ax.set_title('Rearwall Tip (L)')
    elif 'recessed' in label:
        ax = axes[3][0]
        axt = axes[3][1]
        axd = axes[3][2]
        probe = 'recessed'
        ax.set_title('Recessed Tip')
    elif 'sprobe' in label:
        ax = axes[4][0]
        axt = axes[4][1]
        axd = axes[4][2]
        probe = 'sprobe'
        ax.set_title('S Probe Tip')
    else:
        print(f'Probe not matching expected categories, skipping...')
        continue
        
#     angle = np.abs(float(label[-4:]))
    angle = np.round(np.degrees(np.abs(np.squeeze(s.tdata.alphayz))))
    print(angle)

    
    iv_region = np.where(s.iv_data['V'] < -2)
    i = np.where((np.diff(s.iv_data['V'][iv_region]) == 0))
    i = (i[0] + 1, )
    
#     if len(i[0]) / len(iv_region[0]) <= 0.25:
#         print('Big downsample, skipping...')
    i = iv_region
#     s.denormalise()
    
    ax.errorbar(s.iv_data['V'][i], s.iv_data['I'][i], label=angle, yerr=s.iv_data['sigma'][i])
    ax.legend()
    
    try:
        fit_iv_data = iv.IVData.non_contiguous_trim(s.iv_data, i)
        
        if any(len(data_arr) == 0 for variable, data_arr in fit_iv_data.items()):
            print(f'Found empty data array in {label}')
            continue
        fitter = fts.FullIVFitter()
        fit_data = fitter.fit_iv_data(fit_iv_data)
        #     ax.plot(*fit_data.get_fit_plottables())

        fit_df = fit_df.append({'probe': probe, 
                                'angle': angle, 
                                'temp': fit_data.get_temp(),
                                'd_temp': fit_data.get_temp_err(),
                                'isat': fit_data.get_isat(), 
                                'd_isat': fit_data.get_isat_err(),
                                'a': fit_data.get_sheath_exp(), 
                                'd_a': fit_data.get_sheath_exp_err(),
                                'v_f': fit_data.get_floating_pot(),
                                'd_v_f': fit_data.get_floating_pot_err()}, ignore_index=True)

        axt.errorbar(angle, fit_data.get_temp(), yerr=fit_data.get_temp_err(), fmt='x') #, ecolor='silver')
        axd.errorbar(angle, fit_data.get_isat(), yerr=fit_data.get_isat_err(), fmt='x') #, ecolor='silver')
    except RuntimeError as e:
        print(f'WARNING: Encountered error in fit, skipping {label}')

for j, probe in enumerate(['flush', 'angled', 'reecessed', 'rearwall', 'sprobe']):
    filt_df = fit_df.where(fit_df.probe == probe).dropna()
    axt = axes[j][1]
    axd = axes[j][2]
    if j == 0:
        axt.set_title(r'$T_e$')
        axd.set_title(r'$I_{sat}$')
    if j == 1:
        axt.set_xlabel(r'$\theta$')
        axd.set_xlabel(r'$\theta$')
#     axt.errorbar(filt_df['angle'], filt_df['temp'] - 1, yerr=filt_df['d_temp'], fmt='x', ecolor='silver')
#     axt.set_ylim(0.3, 1.7)
    axt.axhline(y=1.0, color='k', linewidth=0.5, linestyle='dotted')
#     axd.errorbar(filt_df['angle'], filt_df['isat'], yerr=filt_df['d_isat'], fmt='x', ecolor='silver')
#     axd.set_ylim(-0.1, 90)
    
fit_df


=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-10.0 ===
10.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-15.0 ===
15.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-2.0 ===
2.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-20.0 ===
20.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-3.0 ===
3.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-30.0 ===
30.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-4.0 ===
4.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-45.0 ===
45.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-5.0 ===
5.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-6.0 ===
6.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-60.0 ===
60.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-7.0 ===
7.0
=== marconi/lowdens_anglescan/rearwall_tiltscan_redo/alpha_yz_-75.0 ===
75.0
=== marconi

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,rearwall,10.0,0.980241,0.010357,33.312793,0.469465,3.025707e-02,0.004540,-3.347164,0.004133
1,rearwall,15.0,1.052761,0.010106,46.672901,0.495541,1.419531e-02,0.003286,-3.115974,0.002593
2,rearwall,2.0,0.734317,0.259680,8.572906,0.122496,8.443950e-15,0.002534,-0.132802,0.696659
3,rearwall,20.0,1.058920,0.030520,58.017129,1.711450,1.676033e-02,0.008999,-3.008013,0.006813
4,rearwall,3.0,0.733604,0.132734,12.819925,0.097740,9.950369e-16,0.001514,0.255650,0.388007
...,...,...,...,...,...,...,...,...,...,...
83,flush,1.0,1.196922,0.159300,2.005588,0.248067,2.345854e-01,0.045250,-2.824577,0.023557
84,flush,70.0,0.998664,0.017359,119.955334,1.635180,1.718279e-02,0.003798,-2.611119,0.003513
85,flush,80.0,1.005111,0.017285,126.388722,1.640139,1.434678e-02,0.003594,-2.533938,0.003454
86,flush,85.0,1.014823,0.019164,129.686942,1.816002,1.130619e-02,0.003869,-2.505975,0.003863


In [20]:
for label, s in splopters.items():
    print(s.iv_data['I'].shape)

(409,)
(550,)
(768,)
(416,)
(501,)
(569,)
(512,)
(402,)
(407,)
(661,)
(658,)
(584,)
(589,)
(511,)
(454,)
(455,)
(455,)
(606,)
(455,)
(364,)
(527,)
(354,)
(656,)
(536,)
(433,)
(437,)
(366,)
(518,)
(422,)
(379,)
(403,)
(455,)
(362,)
(541,)
(516,)
(375,)
(338,)
(657,)
(525,)
(438,)
(375,)
(658,)
(581,)
(363,)
(488,)
(369,)
(606,)
(505,)
(452,)
(357,)
(362,)
(605,)
(583,)
(518,)
(523,)
(662,)
(403,)
(455,)
(367,)
(493,)
(727,)
(373,)
(310,)
(511,)
(661,)
(361,)
(366,)
(611,)
(590,)
(524,)
(528,)
(459,)
(510,)
(408,)
(365,)
(491,)
(455,)
(371,)
(601,)
(606,)
(508,)
(359,)
(586,)
(526,)
(656,)
(390,)
(371,)
(367,)
(526,)
(1085,)


In [16]:
fit_df = fit_df.sort_values(['probe', 'angle']).reset_index(drop=True)
fit_df

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,0.034400,0.008213,-3.372417,0.007165
1,angled,2.0,1.045895,0.016761,7.148155,0.163483,0.013990,0.007510,-3.526380,0.006954
2,angled,3.0,1.048542,0.008816,11.100458,0.129524,0.004756,0.003758,-3.484159,0.003447
3,angled,4.0,1.023927,0.018306,14.362077,0.343829,0.012245,0.007634,-3.423099,0.007020
4,angled,5.0,1.013085,0.010332,18.032248,0.256156,0.010595,0.004569,-3.457255,0.004239
...,...,...,...,...,...,...,...,...,...,...
83,sprobe,30.0,1.033611,0.010399,62.740506,0.552424,0.012586,0.002549,-2.732093,0.002039
84,sprobe,45.0,1.047113,0.014941,83.666200,1.018863,0.007753,0.003510,-2.690400,0.002807
85,sprobe,60.0,1.006582,0.010704,86.495626,0.719475,0.016005,0.002336,-2.594291,0.002069
86,sprobe,75.0,0.996101,0.012401,87.772275,0.795372,0.017997,0.002498,-2.483807,0.002520


In [17]:
fit_data_filename = 'lowdens_fitdata.csv'

In [18]:
# Save dataframe
fit_df.to_csv(fit_data_filename)

## Loading and using the dataframe


In [ ]:
fit_data_filename = 'lowdens_fitdata.csv'